In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
#mean_height = mean(df2[:height])
df2[:height_c] = convert(Vector{Float64}, df2[:height]) # .- mean_height
first(df2, 5)

max_height_c = maximum(df2[:height_c])
min_height_c = minimum(df2[:height_c])

heightsmodel = "
data {
  int N;
  real h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=$(min_height_c),upper=$(max_height_c)> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 0.1);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height_c]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.2s.jls", chn)

chn2 = deserialize("m4.2s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.029, 0.031, 0.029, 0.032) seconds, 0.12 seconds total
Sampling took (0.055, 0.063, 0.059, 0.052) seconds, 0.23 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat
lp__            -1301  2.4e-02    0.99  -1303  -1300  -1300  1.7e+03  7.5e+03  1.0e+00
accept_stat__    0.93  1.3e-03   0.087   0.75   0.96    1.0  4.6e+03  2.0e+04  1.0e+00
stepsize__       0.81  2.6e-02   0.036   0.78   0.81   0.87  2.0e+00  8.7e+00  2.8e+13
treedepth__       2.2  1.3e-02    0.78    1.0    2.0    4.0  3.4e+03  1.5e+04  1.0e+00
n_leapfrog__      5.8  1.1e-01     6.0    1.0    3.0     15  2.9e+03  1.3e+04  1.0e+00
divergent__      0.00      nan    0.00   0.00   0.00   0.00      nan      nan      nan
energy_

end of m4.2s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*